<a href="https://colab.research.google.com/github/Nithya07shree/colab-notes-aiml/blob/main/MLP_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Simple Multi Layer Perception (MLP)

In [1]:
import torch
import torch.nn as nn

class SimpleMLP(nn.Module):
  def __init__(self,input_size, hidden_size, output_size):
    super(SimpleMLP,self).__init__()
    # linear transformation
    self.fc1 = nn.Linear(input_size, hidden_size)
    # Activation function
    self.relu = nn.ReLU()
    # Another linear transformation
    self.fc2 = nn.Linear(hidden_size, output_size)
  def forward(self, X):
    out = self.fc1(X) # z1 = Xw1 + b1
    out = self.relu(out) # a1 = max(0,z1)
    out = self.fc2(out) # z2 = a1w2 + b2
    return out

In [ ]:
# Instantiate model
model = SimpleMLP(784,128,10)
# optimization function from torch. Adam - adaptive moment estimation
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
criterion = nn.crossEntropyLoss()

for images, labels in train_loader:
  # forward pass
  outputs = model(images)
  loss = criterion(outputs, labels)
  # backward pass
  optimizer.zero_grad() # clear gradient
  loss.backward() # calc gradients by backpropagation
  optimizer.step() # update weights

Task:
Use the MNIST (Handwritten Digits) or Fashion-MNIST dataset.
Build a 3-layer MLP in PyTorch (Input $\rightarrow$ Hidden 1 $\rightarrow$ Hidden 2 $\rightarrow$ Output).Use ReLU for hidden layers and CrossEntropyLoss.The Engineering Goal: Achieve >95% accuracy.

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [21]:
# convert images to tensors and normalize it
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)) # Mean and Std Dev of MNIST
])

# containers that actually store the images and labels
train_dataset = datasets.MNIST(root='./data', train = True, download = True, transform = transform)
test_dataset = datasets.MNIST(root='./data', train = False, download = True, transform = transform)

# wrappers around the dataset that controls how data is fed into the model during training and testing loops
train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)


In [23]:
class digitClassifier(nn.Module):
  def __init__(self):
    super(digitClassifier, self).__init__()
    self.flatten = nn.Flatten()
    self.Layers = nn.Sequential(
        # layer 1
        nn.Linear(784,256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        # layer 2
        nn.Linear(256,128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        # layer 3
        nn.Linear(128,10)
    )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.Layers(x)
    return logits

In [24]:
model= digitClassifier()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

def train(epochs=5):
  model.train()
  for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad() # reset gradients
            output = model(data) # train model
            loss = criterion(output, target) # calculate loss
            loss.backward() # calculate gradient
            optimizer.step() # update weights
            total_loss += loss.item()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader)}")

def test():
  model.eval()
  correct=0
  with torch.no_grad():
    for images, labels in test_loader:
      outputs = model(images) # train model
      pred = outputs.argmax(dim=1, keepdim=True)
      correct += pred.eq(labels.view_as(pred)).sum().item()
  accuracy = 100. * correct / len(test_loader.dataset)
  print(f"Test Accuracy: {accuracy}%")

without batch normalization

In [19]:
train(5)
test()

Epoch 1/5, Loss: 0.23123107178569602
Epoch 2/5, Loss: 0.09627869240651682
Epoch 3/5, Loss: 0.0663680920646941
Epoch 4/5, Loss: 0.0491411332943306
Epoch 5/5, Loss: 0.03941924676818124
Test Accuracy: 97.98%


with batch normalization

In [25]:
train(5)
test()

Epoch 1/5, Loss: 0.20320758428662888
Epoch 2/5, Loss: 0.08325019351460898
Epoch 3/5, Loss: 0.059696325836931546
Epoch 4/5, Loss: 0.044100638899603674
Epoch 5/5, Loss: 0.038789263958218634
Test Accuracy: 98.01%
